In [ ]:
from databases import BDS, PosicaoDm1, Bawm,Crm,BaseExtrato,off_Reliance
from funcoes_datas import FuncoesDatas
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date
import numpy as np
import math
from emailer import Email, EmailLer
from pretty_html_table import build_table
from funcoes import CalculosRiscoPort

In [ ]:
#Chama as classes do databases
crm = Crm()
bawm = Bawm()
datas = FuncoesDatas()
extrato = BaseExtrato()
bds = BDS()
fdt = FuncoesDatas()
rel_off = off_Reliance('Primeny')


In [ ]:
##Informações necessárias para rodar a otimização das carteiras- gerar as movimentações

#Tabela com as informações das SC de gestão offshore (nomes, responsáveis e perfil de cada cliente)
po_cadastro = bawm.po_cadastro_all().drop(columns=['CodigoProduto'])

#Puxa da PO_Cadastro os responsaveis por cada supercarteira de gestão (DPMS)

#Lista com os usuários e emails de toda Julius Baer Brazil.
emails_usuarios = crm.lista_usuarios_u()

#Verificar os responsaveis por cada sc de gestão com base no mapa de ativos
responsaveis = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore\Mapa de Ativos/Mapa JBFO_v2.xlsx',sheet_name='Mapa_DB')
responsaveis = responsaveis[(responsaveis['Origem']=='Offshore')& (responsaveis['Tipo de Contrato']!='Distrato em Andamento')][['Conta','SC/ContaMov','RM','Co-RM','Controller','Controller Backup','Segmentação']]

#Altera os usuários u para letra minúscula para padronização (será usado no envio de e-mail)
responsaveis['RM'] =responsaveis['RM'].str.lower()
responsaveis['Co-RM'] =responsaveis['Co-RM'].str.lower()
responsaveis['Controller'] = responsaveis['Controller'].str.lower()
responsaveis['Controller Backup'] = responsaveis['Controller Backup'].str.lower()
#responsaveis['DeputyDPMUsuarioU'] = responsaveis['DeputyDPMUsuarioU'].str.lower()


In [ ]:
# Última data do mes anterior para rodar a query do extrato
data = bds.banco.hoje()
data = datetime.datetime.date(data)

#Traz a alocação ideal por perfil de portfolio
alocacao_por_sc = crm.alocacao_ideial_off()

#Traz os fundos da JBFO (prioridade na aplicação das movimentações)
fundos_jbfo = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Fundos_Casa')
fundos_jbfo = fundos_jbfo[['NomeDoProduto','CodigoProduto','Classificação']]
fundos_jbfo['Fundo']=fundos_jbfo['NomeDoProduto'].str.lower()

#de_paras de mandatos (PI anterior para a atual - equivalência)
mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_mandato')
mandatos = mandatos[['Anterior','Novo']]
#mandatos = dict(mandatos.values)
    
#Traz os deparas de classificação gps para classes de asset Allocation.
dicio_classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacoes=dicio_classificacoes[['Classificação GPS','De Para - Asset Allocation']]
dicio_classificacoes = dict(dicio_classificacoes.values)

#Novos mandatos - Pontos Táticos
novos_mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Novos_mandatos')
novos_mandatos= novos_mandatos[['Portfolio','Classe','Tático']]

#Clientes que estão em distrato
distrato = crm.clientes_em_distrato_off()

#Trazer os produtos GPS e segregar o que é imobiliario (caso precise segregar o que é Reits / ILiquidos)
produtos_gps = crm.produtos_off()
produtos_gps =produtos_gps[['name','new_idmysql','new_tickerid','new_isin','new_idbds','new_idsistemaorigem','new_indgroupaxysidname','transactioncurrencyidname']].rename(columns={'name':'Ativo','new_idmysql':'IdMysqlProduto','new_tickerid':'Ticker','new_isin':'ISIN','new_idsistemaorigem':'CodigoProduto','transactioncurrencyidname':'Moeda'}).reset_index()
produtos_gps['ISIN'] = produtos_gps['ISIN'].str.lower()

#Retorna o nome dos veiculos que fazemos gestao (substituindo a classe JBFO)
dicio_veiculos = dict(pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Fundos_Casa')[['Classe','Veiculo Abreviado']].drop_duplicates().values)



In [ ]:
#Funções usadas na otimização de carteiras

#Retira espaços inúteis
def arruma_espaco(x):
    while x[-1]==' ':
        x=x[:-1]
    return x    
    
#Formatar valor para envio de e-mail
def formatar(valor):
    int_part, dec_part = format(valor, ',.2f').split('.')
    return f"{int_part.replace(',', '.')},{dec_part}"

#Trazer pu do BDS (ainda não usado)
def trazer_pu_atual(id_serie):
    try:
        df = bds.serie_historico(idserie=id_serie, intervalo=1)
        if df.empty:
            df = bds.serie_historico(idserie=id_serie, intervalo=4)
            cota = df.iloc[-1,5]
            ultima_data = df.iloc[-1,3]
            ultima_data =datetime.datetime.date(ultima_data)          
        else:
            df = bds.serie_historico(idserie=id_serie, intervalo=1)
            cota = df.iloc[-1,5]
            ultima_data = df.iloc[-1,3]
            ultima_data =datetime.datetime.date(ultima_data)
    except:
        cota=np.nan
        ultima_data = np.nan
    return cota, ultima_data    

#Função que identifica clientes inativos
def inativo(x):
    x = x.replace('(','').replace(')','')
    x = x.replace(' ','')
    x = x.strip()
    if 'inativ' in x.lower():
        valor='inativo'
    else:
        valor='ok'
    return valor   

In [ ]:
#Consolida a posição atual versus a ideial
alocacao_por_sc = pd.pivot_table(data=alocacao_por_sc, values = 'new_percentual',columns=['new_classedeinvestimentoidname'],index=['new_name','new_conta_supercarteiraidname','new_politicadeinvestimentoidname']).reset_index()
alocacao_por_sc['new_name']=alocacao_por_sc['new_name'].str.lower()
alocacao_por_sc = pd.merge(left = alocacao_por_sc, right = po_cadastro,left_on ='new_conta_supercarteiraidname',right_on = 'NomeSuperCarteiraCRM',how='left')
alocacao_por_sc['inativo']=alocacao_por_sc['new_name'].apply(lambda x: inativo(x))
alocacao_por_sc=alocacao_por_sc[alocacao_por_sc['inativo']!='inativo']
alocacao_por_sc = alocacao_por_sc[['new_name','new_conta_supercarteiraidname','Alt Crédito', 'Alt Multistr','Alt RV L/S', 'Alt Trad/Macro', 'Alt Valor Rel', 'Commodities','Imobiliário', 'Outros', 'P. Equity', 'RF CP', 'RF Glob', 'RF Infl','RF Livre -', 'RF Outros -', 'RV', 'RV Outros -','new_politicadeinvestimentoidname','Titularidade','VetoPowerGestaoTatica','DPM','OfficerUsuarioU','ControllerUsuarioU']].reset_index(drop=True)

In [ ]:
##Traz a posição do último extrato gerado

#Retorna o último dia do mes
ultimo_extrato = data - relativedelta(days=data.day -1)
ultimo_extrato = ultimo_extrato + timedelta(days = -1)

#Gera a carteira da primeira sc da lista
teste = alocacao_por_sc.drop_duplicates()
sc = teste.iloc[0,0]
#sc = '@5AGL1'
df = extrato.posicao_extrato(sc,ultimo_extrato)

#Se o extrato não foi processado no último dia do mês, trazer o último disponível
if df.empty:
    ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
    ultimo_extrato = ultimo_extrato + timedelta(days = -1)
else:
    ultimo_extrato = ultimo_extrato

    
#Gerar a carteira de todas as sc de gestão - portfólios padrão
for i,(index,row) in enumerate(teste.iterrows()):
     if i >0 :
            ultimo_extrato = data
            ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
            ultimo_extrato = ultimo_extrato + timedelta(days = -1)            
            sc = row['new_name']
            df1 = extrato.posicao_extrato(sc,ultimo_extrato)
            if df1.empty:
                ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
                ultimo_extrato = ultimo_extrato + timedelta(days = -1)
            else:
                ultimo_extrato = data
                ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
                ultimo_extrato = ultimo_extrato + timedelta(days = -1)
                ultimo_extrato

            df1 = extrato.posicao_extrato(sc,ultimo_extrato)
            df1['data_ultimo_extrato']=ultimo_extrato          
            df = pd.concat([df,df1])

#Mapeando as informações da base GPS e linkando com os produtos da base (classificação, isin e ticker do CRM).

#Carteira GPS
carteira_gps = df
carteira_gps=pd.merge(left=carteira_gps,right=produtos_gps,on='IdMysqlProduto',how='left')

In [ ]:
#Carteira GPS
carteira_gps = df
carteira_gps=pd.merge(left=carteira_gps,right=produtos_gps,on='IdMysqlProduto',how='left')

In [ ]:
#Fazer a conciliação de contas
conciliacao = df[['NomeContaCrm','NomeSupercarteiraCrm']].drop_duplicates()
conciliacao=conciliacao['NomeSupercarteiraCrm'].str.lower()
todas_sc = alocacao_por_sc[['new_name','new_conta_supercarteiraidname']].drop_duplicates()
conciliacao = pd.merge(left=conciliacao, right=todas_sc,left_on='NomeSupercarteiraCrm',right_on='new_name',how='right')
conciliacao = conciliacao[conciliacao['NomeSupercarteiraCrm'].isnull()]
conciliacao.to_excel('conciliacao.xlsx')

In [ ]:
#Ajustes nas - futuratmente tirar colunas inúteis
carteira_gps = carteira_gps[['DataPosicao','NomeSupercarteiraCrm','NomeContaCrm','NomeDoProduto','CodigoProduto_x','ISIN','QuantidadeCotas','ValorCota','SaldoNaData','Moeda','Classe', 'Subclasse']].rename(columns={'DataPosicao':'Data_Posicao','NomeContaCrm':'Nome_Cliente','NomeDoProduto':'Nome_produto','QuantidadeCotas':'quantidade','ValorCota':'PU','CodigoProduto_x':'CodigoProduto'})
carteira_gps['Esteira']='ex-Gps'
carteira_gps['Nome_produto']=carteira_gps['Nome_produto'].str.lower().apply(lambda x: arruma_espaco(x))


In [ ]:
#Concatenando as carteiras com a custódia - GPS

sc_axys =  crm.axys__sc()
prod_axys = crm.ays_produtos()

import os
caminho = "I:/Shared/Processamento_Conciliacao_Offshore/CARTEIRAS/AXYS/"
lista_arquivos = os.listdir(caminho)
    
lista_datas = []
for arquivo in lista_arquivos:
# descobrir a data desse arquivo
    if ".xlsm" in arquivo:
        datas = os.path.getmtime(f"{caminho}/{arquivo}")
        lista_datas.append((datas, arquivo))    
        lista_datas.sort(reverse=True)
        ultimo_arquivo = lista_datas[0]
        ultimo_arquivo = ultimo_arquivo[1].replace('~$','')
custodias = pd.read_excel(caminho+ultimo_arquivo,sheet_name = 'Axys',header = None)
colunas = custodias.iloc[1]
custodias.columns = colunas
custodias = custodias.drop(index=[0,1])
custodias = custodias[['Portfólio','Security Simbol','Banco']]
colunas = custodias.columns
custodias[colunas] = custodias[colunas].apply(lambda x: x.str.replace(',', '.').astype('str'))
custodias = pd.merge(left =custodias,right=sc_axys,left_on='Portfólio',right_on='new_name',how='left').drop(index=[0,1]).drop(columns={'new_name','Portfólio'})[['new_supercarteiraoriginadoraidname','Security Simbol','Banco']]
custodias = pd.merge(left =custodias,right=prod_axys,left_on='Security Simbol',right_on='new_axyssecuritysymbol',how='left')
carteira_gps = pd.merge(left =carteira_gps,right=custodias,left_on=['NomeSupercarteiraCrm','CodigoProduto'],right_on=['new_supercarteiraoriginadoraidname','new_idsistemaorigem'],how='left').drop(columns={'new_supercarteiraoriginadoraidname','Security Simbol','name','new_idsistemaorigem','new_axyssecuritysymbol'}).rename(columns={'productid':'guidproduto','Banco':'Custódia'})

In [ ]:
#Trazer a carteira da Reliance

#Retorna todos os produtos base Reliance
produtos_gestao = rel_off.buscar_fundos_jbfo()

#Traz a recomendação de portfolio por cliente
recomendacao = pd.read_excel('port_rec.xlsx')
recomendacao = recomendacao[['Cliente','Mandato_Recomendado']]

#Puxando a ultima data do mes anterior para calculo do extrato
data_rel_cart = datetime.datetime.date(fdt.hoje() - relativedelta(days=fdt.hoje().day -1)+timedelta(days=-1))

# #Mapeando as informações da base Reliance e linkando com os produtos da base (classificação, isin e ticker do CRM).

#Trazer a última posição disponivel (mes anterior)
carteira_rel = rel_off.carteira_reliance(data_rel_cart)
trazer_data = datetime.datetime.date(carteira_rel['calculado_em'].max())
carteira_rel['calculado_em'] = carteira_rel['calculado_em'].apply(lambda x : datetime.datetime.date(x))
carteira_rel = carteira_rel[carteira_rel['calculado_em'] == trazer_data].rename(columns= {'asset_class':'Subclasse','moeda':'Moeda'}).drop(columns={'Subcategoria'})
carteira_rel = carteira_rel[~(carteira_rel['Nome_produto'].str.contains('FWD'))&(carteira_rel['SaldoNaData']>=0)].drop(columns={'calculado_em'})

#Incluir os daldos na composição da carteira
saldos = rel_off.buscar_saldos()

saldos['ISIN'] = 'caixa'
saldos = saldos[['Data_Posicao', 'Nome_Cliente', 'cd_security', 'ISIN','Nome_produto', 'Classe', 'Subclasse', 'quantidade','PU', 'cotacao', 'SaldoNaData', 'Moeda']]

#Concatenando as carteiras com saldos
carteira_rel = pd.concat([carteira_rel,saldos]).sort_values(by=['Nome_Cliente'])

#Ajuste da Classe para padrão GPS
classes_padrao = pd.read_excel('classificacao.xlsx')[['ID_Produto','cd_asset_class_name']]
carteira_rel = pd.merge(left = carteira_rel, right=classes_padrao,left_on='cd_security', right_on='ID_Produto', how='left')
carteira_rel['Classe'] = carteira_rel['cd_asset_class_name']
carteira_rel = carteira_rel.drop(columns={'ID_Produto','cd_asset_class_name'})

#Remover todos os valores iguais a 0

carteira_rel = carteira_rel.dropna(subset=['SaldoNaData'])

#Ajusta as colunas
carteira_rel['NomeSupercarteiraCrm'] = carteira_rel['Nome_Cliente']
carteira_rel['Esteira'] = 'ex-Reliance'
carteira_rel['CodigoProduto'] = carteira_rel['cd_security'].astype('str') 
carteira_rel = carteira_rel.rename(columns={'moeda':'Moeda'})
carteira_rel = carteira_rel[['Data_Posicao', 'NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto','CodigoProduto', 'ISIN', 'quantidade', 'PU', 'SaldoNaData', 'Moeda','Classe', 'Subclasse', 'Esteira']]


In [ ]:
#Concatenando as carteiras com a custódia da Reliance

caminho = "I:/Shared/Processamento_Conciliacao_Offshore/CARTEIRAS/SIS INTERNACIONAL/"
lista_arquivos = os.listdir(caminho)
    
lista_datas = []
for arquivo in lista_arquivos:
# descobrir a data desse arquivo
    if ".xls" in arquivo:
        datas = os.path.getmtime(f"{caminho}/{arquivo}")
        lista_datas.append((datas, arquivo))    
        lista_datas.sort(reverse=True)
        ultimo_arquivo = lista_datas[0]
        ultimo_arquivo = ultimo_arquivo[1].replace('~$','')
custodias = pd.read_excel(caminho+ultimo_arquivo,sheet_name = 'ToExcel_Posicoes',header = None)
colunas = custodias.iloc[0]
custodias.columns = colunas
custodias = custodias.drop(index=[0])
custodias = custodias[['CLIENTE','ATIVO','cd_security','INSTITUIÇÃO']]
colunas = custodias.columns
custodias[colunas] = custodias[colunas].apply(lambda x: x.replace(',', '.').astype('str'))
carteira_rel = pd.merge(left =carteira_rel,right=custodias,left_on=['NomeSupercarteiraCrm','CodigoProduto'],right_on=['CLIENTE','cd_security'],how='left').drop(columns={'CLIENTE','ATIVO','cd_security'}).rename(columns ={'INSTITUIÇÃO':'Custódia'})
carteira_rel['guidproduto'] = 'Rel '+ carteira_rel['CodigoProduto']

In [ ]:
carteira_jbfo = pd.concat([carteira_rel,carteira_gps])
carteira_custodia = carteira_jbfo.to_excel('carteira_jbfo.xlsx')

In [ ]:
#Concatenar as carteiras
carteira_jbfo = pd.concat([carteira_rel,carteira_gps])
carteira_jbfo['ISIN'] = carteira_jbfo['ISIN'].str.lower()

#Filtra os ativos que são imobiliarios e gera uma planilha para consultar na bbg o que é Reit
ativos_imob = carteira_jbfo[carteira_jbfo['Classe']=='Imobiliário'][['Nome_produto','ISIN']]
ativos_imob.to_excel('I:/Shared/SAO_Investimentos/Portfolio Management/Tamara/verificar_reits.xlsx')

#Traz a planilha com as infos da bbg
classificacao_imob = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/Base Real Estate.xlsx')
classificacao_imob['ISIN'] = classificacao_imob['ISIN'].str.lower()

#Separando os Reits dos Iliquidos
carteira_jbfo = pd.merge(left=carteira_jbfo,right=classificacao_imob,on='ISIN',how='left').drop(columns=['Nome_produto_y']).rename(columns={'Tipo':'Classificacao_Real_Estate'})
carteira_jbfo['Classe'].loc[carteira_jbfo['Classe']=='Imobiliário'] = carteira_jbfo['Classificacao_Real_Estate'].loc[carteira_jbfo['Classe']=='Imobiliário']
carteira_jbfo = carteira_jbfo.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto_x', 'ISIN', 'quantidade','PU']).sort_values(by=['Esteira'])

#Ajuste das classes (padronização dos nomes das classes) - tirar pontos nas classes("-")
carteira_jbfo['Classe']=carteira_jbfo['Classe'].astype('str')
classes = carteira_jbfo['Classe']
carteira_jbfo['Classe']= [x.replace(' -','') for x in classes]

#DEixando apenas as colunas necessárias, renomeando as mesmas com nomes mais intuitivos
carteira_jbfo = carteira_jbfo[['Data_Posicao','NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto_x','CodigoProduto', 'ISIN', 'quantidade','PU','SaldoNaData', 'Moeda', 'Classe', 'Subclasse', 'Esteira']].rename(columns={'Classe':'classe_sistema', 'Subclasse':'subclasse_sistema','Nome_produto_x':'Nome_produto'})
carteira_jbfo = pd.merge(left = carteira_jbfo, right = po_cadastro,right_on ='NomeContaCRM',left_on = 'Nome_Cliente',how='left')
carteira_jbfo = pd.merge(left = carteira_jbfo, right = alocacao_por_sc,right_on ='new_conta_supercarteiraidname',left_on = 'Nome_Cliente',how='left')
carteira_jbfo=carteira_jbfo[['Data_Posicao','NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto','CodigoProduto', 'ISIN', 'quantidade',
        'PU','SaldoNaData', 'Moeda', 'classe_sistema', 'subclasse_sistema', 'Esteira',
        'new_politicadeinvestimentoidname','Officer', 'Controller']].rename(columns={'new_politicadeinvestimentoidname':'Politica'})
carteira_jbfo = carteira_jbfo.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'ISIN', 'quantidade','PU']).sort_values(by=['Esteira'])

#Preenchendo colunas com NA com estimações
carteira_jbfo['Politica'] = carteira_jbfo['Politica'].fillna('Pontos de Risco')
carteira_jbfo['Officer'] =  carteira_jbfo['Politica'].fillna('Tamara Alves')
carteira_jbfo['Controller'] = carteira_jbfo['Controller'].fillna('Tamara Alves')


In [ ]:
#Cria o dataframe de movimentos, retira os produtos que não possuem classe cadastrada e faz depara com a classes de asset allocation.
carteira_jbfo['classe_sistema'] = carteira_jbfo['classe_sistema'].fillna('Outros').replace('nan','Outros')

#Igualando as classificações dos produtos
carteira_jbfo = carteira_jbfo.drop_duplicates()
carteira_jbfo['classe_sistema']=carteira_jbfo['classe_sistema'].apply(lambda x:dicio_classificacoes[x])

#Retornando os produtos da JBFO
carteira_jbfo = pd.merge(left = carteira_jbfo, right = produtos_gestao,left_on= 'Nome_produto',right_on ='security',how='left')

#Identifica quais produtos são fundos gestão JBFO - 
carteira_jbfo=pd.merge(left=carteira_jbfo,right=fundos_jbfo,on='CodigoProduto',how='left')
carteira_jbfo['Classificação'] = carteira_jbfo['Classificação'].astype('str')
carteira_jbfo['Classificação'].loc[carteira_jbfo['cd_security'].notnull()] = carteira_jbfo['classe_sistema'] + ' - JBFO'
carteira_jbfo['Classificação'] = carteira_jbfo.apply(lambda row : 'Alternativos Macro Arbitragem - MS - JBFO' if 'H3 COMPOSITE' in row['Nome_produto'] else row['Classificação'],axis=1)


carteira_jbfoa = carteira_jbfo[['Data_Posicao','NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto','CodigoProduto', 'ISIN_x', 'quantidade',
        'PU','SaldoNaData', 'Moeda', 'classe_sistema', 'subclasse_sistema', 'Esteira',
        'Politica','Officer', 'Controller']].rename(columns={'ISIN_x':'ISIN'})

carteira_jbfo['classe_sistema']=carteira_jbfo.apply(lambda row: row['classe_sistema'] if row['Classificação']=='nan' else row['Classificação'],axis=1 )


In [ ]:
#Encontrar a nova tapi (indicada para cada cliente de acordo com a tapi anterior) - retira as duplicidades.
carteira_jbfo = pd.merge(left=carteira_jbfo,right=mandatos,left_on='Politica',right_on='Anterior',how='left')
carteira_jbfo = pd.merge(left=carteira_jbfo,right=recomendacao,left_on='Nome_Cliente',right_on='Cliente',how='left')
carteira_jbfo['Novo'].loc[carteira_jbfo['Novo'].isnull()] = carteira_jbfo['Mandato_Recomendado'].loc[carteira_jbfo['Novo'].isnull()]
carteira_jbfo=carteira_jbfo.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','PU']).drop(columns={'Cliente','Mandato_Recomendado'})
carteira_jbfo['Anterior'] = np.where((carteira_jbfo['Esteira']=='ex-Reliance')&(carteira_jbfo['Anterior'].isnull()),'Politica_Pontos',carteira_jbfo['Anterior'])
carteira_jbfo =carteira_jbfo.dropna(subset=['Novo'])


In [ ]:
#Verifica se os clientes possuem H3 Strategy caso sim, eles precisam ter o mandato que some Alt LS + Alt. Crédito + Alt. Multiestrategia (MS)
carteira_jbfo['Possui_H3']= carteira_jbfo.apply(lambda row: 1 if row['classe_sistema']=='Alternativos Macro Arbitragem - MS - JBFO' else 0,axis=1)

#Calcula o percentual do produto dentro da carteira, o financeiro total e desconsiderando os ativos iliquidos.
percentual_produto=[]
financeiro_total_spe =[]
financeiro_total =[]
mandato = []

for ind,row in carteira_jbfo.iterrows():   
        h3=0
        #Soma de todos os ativos
        soma = carteira_jbfo[carteira_jbfo['NomeSupercarteiraCrm']==row['NomeSupercarteiraCrm']]['SaldoNaData'].sum()
        financeiro_total.append(soma)
        #Soma dos ativos exceto iliquidos
        iliquidos = carteira_jbfo[(carteira_jbfo['NomeSupercarteiraCrm']==row['NomeSupercarteiraCrm'])&(carteira_jbfo['classe_sistema']=='Iliq. P. Equity')]['SaldoNaData'].sum()
        financeiro_total_spe.append(soma-iliquidos)
        #Percentual de cada produto (do financeiro sem contar com os ativos iliquidos)    
        percentual= row['SaldoNaData']/(soma-iliquidos)
        percentual_produto.append(percentual)
        #Somatorio das classes Alt LS + Alt. Crédito + Alt. Multiestrategia para clientes que possuem H3 Strategy.
        h3 = carteira_jbfo[(carteira_jbfo['Nome_Cliente']==row['Nome_Cliente'])&(row['Novo'][-2:]!='MS')]['Possui_H3'].sum()
        if h3>0:
            mandato_fundo = row['Novo']+' MS'
        else:
            mandato_fundo = row['Novo']
        mandato.append(mandato_fundo)        
    
#Incluir no dataframe as informações geradas acima
carteira_jbfo['percentual_produto']=percentual_produto
carteira_jbfo['Financeiro_Total']=financeiro_total
carteira_jbfo['Financeiro_Total_s/pe']=financeiro_total_spe
carteira_jbfo['Portfolio']=mandato                                            
carteira_jbfo = carteira_jbfo.sort_values(by=['classe_sistema'])
carteira_jbfo = carteira_jbfo.fillna(0)
carteira_jbfo=carteira_jbfo.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','SaldoNaData']).drop(columns=['Possui_H3'])
carteira_jbfo.to_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/8 - Carteiras Off/carteira_off_jbfo.xlsx')



In [ ]:
#Deixando apenas as colunas que interessam para o groupby - Agrupando por classe de ativo para calculo de movimentação para chegar no ponto atual dos novos mandatos
carteira_jbfo = carteira_jbfo.groupby(['Data_Posicao','Nome_Cliente','NomeSupercarteiraCrm','classe_sistema','Anterior','Portfolio','Financeiro_Total','Financeiro_Total_s/pe']).sum().reset_index()

In [ ]:
#Traz todos os clientes que deverão ser movimentados
carteira_jbfo = carteira_jbfo[['Data_Posicao', 'Nome_Cliente', 'NomeSupercarteiraCrm',
       'classe_sistema', 'Anterior', 'Portfolio', 'Financeiro_Total',
       'Financeiro_Total_s/pe', 'quantidade', 'PU', 'SaldoNaData','percentual_produto']]

clientes = carteira_jbfo['NomeSupercarteiraCrm']
classes = novos_mandatos['Classe'].drop_duplicates()
#clientes = ['S HOT']

#Traz as informações por cliente (nomes, valores da posição com e sem iliquidos,o percentual atual e proposto de cada classe nos novos portfolios)
dataframe = []    
for classe in classes:
    lista = []
    for cliente in clientes:  
        carteira_jbfo_selecao=carteira_jbfo[carteira_jbfo['NomeSupercarteiraCrm']==cliente]
        Data_posicao = carteira_jbfo_selecao.iloc[0,0]
        Nome_Cliente = carteira_jbfo_selecao.iloc[0,1]
        NomeSupercarteiraCrm = cliente
        classe_sistema = classe
        #Retorna o mandato anterior
        Anterior = carteira_jbfo_selecao.iloc[0,4]
        #Retorna o mandato atual
        Portfolio = carteira_jbfo_selecao.iloc[0,5]
        Financeiro_Total = carteira_jbfo_selecao.iloc[0,6]
        Financeiro_Total_pe = carteira_jbfo_selecao.iloc[0,7]
        #Traz o saldo de cada cliente por classe
        try:
            Saldo = carteira_jbfo_selecao[carteira_jbfo_selecao['classe_sistema']==classe_sistema].iat[0,10]
        except:
            Saldo = 0
        try:
            percentual_produto = carteira_jbfo_selecao[carteira_jbfo_selecao['classe_sistema']==classe_sistema].iat[0,11]
        except:
            percentual_produto = 0
        #Traz o percentual proposto no novo mandato para cada classe    
        percentual_proposto= novos_mandatos[(novos_mandatos['Classe']==classe)& (novos_mandatos['Portfolio']==Portfolio)].iat[0,2]
        
        #Preenche o dataframe as informações acima (formato dicionario)
        dataframe.append({'Data_Posicao':Data_posicao,'Nome_Cliente': Nome_Cliente,'NomeSupercarteiraCrm':NomeSupercarteiraCrm, 'classe_sistema': classe_sistema,'PI_Anterior':Anterior,'PI_Proposta':Portfolio,'Financeiro_Total':Financeiro_Total,'Financeiro_s/PE':Financeiro_Total_pe,'Financeiro_Produto':Saldo,'Posicao_Atual':percentual_produto,'Posicao_Proposta':percentual_proposto})  

#Gera um novo dataframe com os dados tratados anterioremente mais os percentuais atuais e os percentuais recomendados nos novos portfolios.       
carteira_jbfo = pd.DataFrame.from_dict(dataframe)
carteira_jbfo['Movimento']=(carteira_jbfo['Posicao_Proposta']-carteira_jbfo['Posicao_Atual'])*carteira_jbfo['Financeiro_s/PE']            
movimentos_gps=carteira_jbfo.fillna(0)


##Deixa em pivot table os dados para calculo dos valores financeiros das movimentações
#Fazendo um pivot table e configurando o mesmo.
carteira_jbfo = pd.pivot_table(data=carteira_jbfo,values = ['Posicao_Atual','Posicao_Proposta','Movimento'],columns=['classe_sistema'],index=['Data_Posicao','Nome_Cliente','NomeSupercarteiraCrm','PI_Anterior','PI_Proposta','Financeiro_Total','Financeiro_s/PE'])
carteira_jbfo = carteira_jbfo.reset_index()
carteira_jbfo.columns = carteira_jbfo.columns.droplevel(0)

#Configurando as colunas para segregar o que é o percentual atual para o proposto
inicio = ['Data_Posicao','SC_Gestão','SC_OFF','Mandato_Anterior','Mandato_Proposto','Financeiro_Total','Financeiro_s/PE']
classe_atual =list(carteira_jbfo.columns[-20:])
classes_proposta=list([str(i)+"-Proposto" for i in classes])
classes_movimento=list([str(i)+"-movimento" for i in  classes])
carteira_jbfo.columns = inicio + classes_movimento+classe_atual+classes_proposta



In [ ]:
#Encontrando as respectivas 

In [ ]:
#Fazendo o groupby para geração dos movimentos

#Soma a classe de Alt.LS Asia com Alt.LS
carteira_jbfo['Alternativos Long Short']=carteira_jbfo['Alternativos Long Short']+carteira_jbfo['Alternativos Long Short Asia - JBFO']

In [ ]:
##Gerando os valores financeiros de cada classe.
classes_jbfo = fundos_jbfo['Classificação'].drop_duplicates().to_list()
classes_jbfo.remove('Alternativos Macro Arbitragem - MS - JBFO')
classes_jbfo.remove('Alternativos Long Short Asia - JBFO')

##Segregando o que é veiculo JBFO (Tudo o que está JBFO é o saldo da classe em produtos geridos por nós) e calculando os valores a aplicar/receber em cada classe
#classes_jbfo : Retorna os valores dos veiculos de nossa gestão por classe
#valor_veiculo : o que movimentaremos no veiculo, aplicação sempre prioriza fundos da casa, em caso de resgate prioriza outros produtos.

for classe in classes_jbfo:
    lista = []
    lista2 = []
    for ind,row in carteira_jbfo.iterrows():
        cliente = row['SC_OFF']
        posicao_classe = row[classe[:-7]]
        posicao_esperada = row[classe[:-7]+'-Proposto']
        posicao_veiculo =  row[classe]
        posicao_classe = posicao_classe + posicao_veiculo
        movimento = posicao_esperada -  posicao_classe
        financeiro = row['Financeiro_s/PE']
        if  movimento>0:
            valor_veiculo = movimento*financeiro
            valor_outros = 0
        elif (movimento < 0) and (posicao_classe*0.95*-1)>movimento:
            valor_veiculo = posicao_veiculo*financeiro*-1
            valor_outros = (posicao_classe-posicao_veiculo)*financeiro*-1
        elif (movimento <0) and (posicao_classe-posicao_veiculo)>(movimento*-1):
            valor_veiculo = 0
            valor_outros = movimento*financeiro
        elif (movimento <0) and ((posicao_classe-posicao_veiculo)==0):
            valor_veiculo = movimento*financeiro
            valor_outros = 0            
        elif row['Alternativos Long Short Asia - JBFO']>0:
            valor_veiculo = (posicao_esperada -posicao_classe)*financeiro
            valor_outros = 0    
        else:
            valor_veiculo = (posicao_esperada-posicao_classe+(posicao_classe-posicao_veiculo))*financeiro
            valor_outros = (movimento*financeiro)- valor_veiculo         
        lista.append(valor_veiculo)
        lista2.append(valor_outros)

    carteira_jbfo[classe+'-movimento'] = lista
    carteira_jbfo[classe[:-7]+'-movimento'] = lista2
    
# Unifica o veiculo JBFO de Reits aos outros ativos de mesma classe      
carteira_jbfo['Real Estate-movimento'] = carteira_jbfo['Real Estate-movimento']+ carteira_jbfo['Real Estate - JBFO-movimento'] 

#Gera excel com todos os movimentos propostos
carteira_jbfo.to_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/8 - Carteiras Off/movimentos.xlsx')

In [ ]:
# #Localiza os emails dos responsaveis por cada sc e retira da lista os clientes que estão distrato

carteira_jbfo = pd.merge(left=carteira_jbfo,right=responsaveis,left_on='SC_OFF',right_on='SC/ContaMov',how='left')
carteira_jbfo = pd.merge(left=carteira_jbfo,right=distrato,left_on='SC_OFF',right_on='new_name',how='left')
carteira_jbfo = carteira_jbfo[carteira_jbfo['new_contratoname']!='Distrato em Andamento']
carteira_jbfo = carteira_jbfo.drop(columns=['Conta','SC/ContaMov','new_name','new_contrato','new_contratoname']).drop_duplicates()

#Substitui os campos controllers, dpms que estão em branco para o PM off.
carteira_jbfo['RM'] = carteira_jbfo['RM'].fillna('tamara alves')
carteira_jbfo['Co-RM'] = carteira_jbfo['Co-RM'].fillna('tamara alves')
carteira_jbfo['Controller'] = carteira_jbfo['Controller'].fillna('tamara alves')
carteira_jbfo['Controller Backup'] = carteira_jbfo['Controller Backup'].fillna('tamara alves')

#Retorna os emails dos responsáveis por cada carteira
emails_envio = []
emails_usuarios['fullname']=emails_usuarios['fullname'].str.lower()
for idx, row in carteira_jbfo.iterrows(): 
    lista_email=[]
    if row['RM'] in emails_usuarios['fullname'].to_list():  
        lista_email.append(emails_usuarios.loc[emails_usuarios['fullname']== row['RM']].iat[0,2])
    if row['Co-RM'] in emails_usuarios['fullname'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['fullname']==row['Co-RM']].iat[0,2])
    if row['Controller'] in emails_usuarios['fullname'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['fullname']==row['Controller']].iat[0,2])
    if row['Controller Backup'] in emails_usuarios['fullname'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['fullname']==row['Controller Backup']].iat[0,2]) 
    emails_envio.append(lista_email)
carteira_jbfo['emails_aprovadores']= emails_envio        

In [ ]:
#Retira das colunas os nomes 'movimento' e 'proposto' pois não são necessários para o envio do e-mail de movimentação)
colunas = carteira_jbfo.columns
colunas = [col.replace('-movimento','') for col in colunas]
colunas = [col.replace('-Proposto','') for col in colunas]
carteira_jbfo.columns = colunas

#Retira as classes que não conseguimos movimentar por questões diversas (tamanho, etc...)
carteira_jbfo = carteira_jbfo.drop(columns={'Iliq. P. Equity','Alternativos Long Short Asia','Alternativos Long Short Asia - JBFO','Real Estate - JBFO'})

In [ ]:
officers = movimentos_gps['RM'].drop_duplicates().to_list()
#Teste = officers = ['bruna casanova']
classes_pm = list(movimentos_gps.columns[7:23])


##Gera o email que será enviado para cada officer

#Puxa os emails dos destinatários responsáveis pela carteira.
for officer in officers:
    df = movimentos_gps[movimentos_gps['RM']==officer]
    clientes =df['SC_Gestão']
    sc_off = df['SC_OFF']
    txt_email=[]
    emails = [x for i,elem in df['emails_aprovadores'].items() for x in elem]
    emails=list(set(emails))
    emails = [x for x in emails if x is not None]
    
    txt_inicio = f'''Prezados,<br>   
        Seguem as movimentações propostas para adequação dos portfólios atuais para os novos mandatos. Caso tenham alguma sugestão vinda da equipe dos DPMs, favor desconsiderar.<br><br>    
        Os portfólios que possuem "MS", são aqueles que possuem o H3 Composite (classe Multiestratégia)-<br><br>
        ''' 
#Gera os movimentos que cada sc precisa fazer.
    for sc in sc_off:
        
        df = movimentos_gps[movimentos_gps['RM']==officer]
        df = df[df['SC_OFF']==sc]
        df[df['SC_OFF']==sc][['SC_Gestão']].iloc[0,0]
        if df.empty:
            txt = txt1 = f'<ul><li>Sem movimentações a fazer.</li>'
        else:    
            mandato_atual = df['Mandato_Anterior'].iloc[0]
            mandato_proposto = df['Mandato_Proposto'].iloc[0]
            patrimonio_total = formatar(df['Financeiro_Total'].iloc[0])
            patrimonio_s_pe = formatar(df['Financeiro_s/PE'].iloc[0]) 
            df_financeiro= df.iloc[:, 0:23]
            df_atual= df.iloc[:, 23:39]
            df_proposto=df.iloc[:, 39:55]   
            SC_On = df['SC_Gestão'].iloc[0]
            data_posicao=df.iat[0,0]
            data_posicao = "{}/{}/{}".format(data_posicao.day,data_posicao.month,data_posicao.year)
            txt = f'''<b>Cliente {sc} / {SC_On}:</b><br>
            *Data Posição : {data_posicao},<br>
            * Mandato : {mandato_atual},<br>
            * Mandato Proposto : {mandato_proposto},<br>
            * PL Total : {patrimonio_total},<br>
            * PL desconsiderando Ilíquidos e Imobiliarios iliquidos : {patrimonio_s_pe},<br>'''
            for classe_pm in classes_pm:     
                if classe_pm == 'Alternativos Macro Arbitragem - MS - JBFO':
                    classe_pm_aloc = 'Alternativos Macro Arbitragem - MS - JBFO'
                    ativo = 'H3 Composisite'
                elif classe_pm[-7:]==' - JBFO':
                    classe_pm_aloc = classe_pm[:-7]
                    ativo = dicio_veiculos[classe_pm]
                else:    
                    classe_pm_aloc=classe_pm
                    ativo = dicio_veiculos[classe_pm]            
                percentual_proposto_valor=df_proposto[classe_pm_aloc].iloc[0]*100
                try:
                    percentual_atual_valor=(df_atual[classe_pm_aloc].iloc[0]+df_atual[classe_pm_aloc+' - JBFO'].iloc[0])*100
                except:
                    percentual_atual_valor=df_atual[classe_pm_aloc].iloc[0]*100


                #                 percentual = novos_mandatos[(novos_mandatos['Classe']==classe)&(novos_mandatos['Portfolio']==mandato_proposto)].iat[0,2]*100
                #             else:
                #                 classe=classe_pm
                #                 percentual = novos_mandatos[(novos_mandatos['Classe']==classe)&(novos_mandatos['Portfolio']==mandato_proposto)].iat[0,2]*100
                try:
                    valor = df_financeiro[classe_pm].iloc[0]
                    valor = round(valor,-4)
                    classe = classe_pm
                except:
                    valor=0            
                percentual_atual_valor ='{0:.2f}%'.format(percentual_atual_valor)
                percentual_proposto_valor ='{0:.2f}%'.format(percentual_proposto_valor)
                if valor < 0:
                    valor = abs(valor) 
                    valor=formatar(valor)
                    txt1 = f'<ul><li>Resgate de ${valor} de {ativo} para levar a alocação da classe de {percentual_atual_valor} para o ponto tático de {percentual_proposto_valor}.</li>'
                elif valor > 0:
                    valor = abs(valor)
                    valor=formatar(valor)
                    txt1 = f'<ul><li>Aplicação de ${valor} de {ativo} para levar a alocação da classe de {percentual_atual_valor} para o ponto tático de {percentual_proposto_valor}.</li>'
                else:
                    txt1=''
                txt += txt1 + '</ul>'

            txt_email.append(txt)
        corpo = f"{txt_inicio}{'<br>'.join(txt_email)}"

    subject= f'Movimentos offshore - Novos Mandatos - Base ex-GPS'
    to=['tamara.alves@jbfo.com','portfolio@jbfo.com']+emails
    email = Email(to = to , subject = subject, text= corpo,send = True)    
    
#Gera o excel de cada officer.
    df = df.reset_index(drop=True)
    df= df.iloc[:, 0:23]
    df.to_excel(f'O:/SAO/\CICH_All\/Portfolio Management/20 - Offshore/1 - Movimentao_officer/{officer}.xlsx')
    






In [ ]:
carteira = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/8 - Carteiras Off/carteira_off_jbfo.xlsx')
posicao_clopps = carteira[(carteira['Nome_produto']=='cl multi-strategy fund - opportunities segregated portfolio (usd)')| (carteira['Nome_produto']== 'CL MULTI-STRATEGY FUND SICAV-SIF - OPPORTUNITIES')][['Nome_Cliente','SaldoNaData','percentual_produto']]
posicao_rflivre = carteira[(carteira['classe_sistema']=='Renda Fixa Crédito - JBFO')|(carteira['classe_sistema']=='Renda Fixa Crédito')][['NomeSupercarteiraCrm','Nome_Cliente','Anterior','Novo','Financeiro_Total','Financeiro_Total_s/pe','classe_sistema','SaldoNaData','percentual_produto']]
posicao_rflivre = pd.pivot_table(data=posicao_rflivre,values = ['SaldoNaData','percentual_produto'],columns=['classe_sistema'],index=['NomeSupercarteiraCrm','Nome_Cliente','Anterior','Novo','Financeiro_Total','Financeiro_Total_s/pe'],aggfunc=["sum"]).reset_index()
posicao_rflivre.columns = posicao_rflivre.columns.droplevel(0)
posicao_rflivre.columns=['SC','Nome_Cliente','Mandato_Anterior','Mandato_Proposto','Financeiro_Total','Financeiro_Total_s/pe','Ativos_Credito_Outros','Fundos_JBFO_Cred','%Ativos_Credito','%Fundos_JBFO_Cred']
posicao_rflivre['Total_Credito'] = posicao_rflivre['Ativos_Credito_Outros']+posicao_rflivre['Fundos_JBFO_Cred']
posicao_rflivre['%_Credito'] = posicao_rflivre['Total_Credito']/posicao_rflivre['Financeiro_Total_s/pe']
movimento_clopps = pd.merge(left = posicao_rflivre, right = posicao_clopps,on = 'Nome_Cliente',how='left').rename(columns={'SaldoNaData':'Posicao_Clopps'}).fillna(0)

#TRatando os movimentos propostos e vendo se é do Clopps
movimentos_proposta = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/8 - Carteiras Off/movimentos.xlsx')
movimentos_proposta = movimentos_proposta[['SC_OFF','Renda Fixa Crédito-movimento','Renda Fixa Crédito - JBFO-movimento','Renda Fixa Crédito-Proposto']].rename(columns={'SC_OFF':'Nome_Cliente'})

movimento_clopps = pd.merge(left = movimento_clopps, right = movimentos_proposta,on = 'Nome_Cliente',how='left')

In [ ]:
movimento_clopps = movimento_clopps[['SC','Mandato_Anterior','Mandato_Proposto','Financeiro_Total','Financeiro_Total_s/pe','Total_Credito','Ativos_Credito_Outros','Fundos_JBFO_Cred','Posicao_Clopps','Renda Fixa Crédito-movimento','Renda Fixa Crédito - JBFO-movimento','%_Credito','percentual_produto','Renda Fixa Crédito-Proposto']].rename(columns = {'percentual_produto':'Percentual_Clopps'})

In [ ]:
movimento_clopps.to_excel('analise_clops.xlsx')